In [29]:
# Usual Libraries
import pandas as pd
import numpy as np
import seaborn as sns

# Machine Learning
import sklearn
from sklearn import preprocessing

# Librosa (the mother of audio files)
import librosa
import librosa.display
import IPython.display as ipd
import warnings
import os

# Primera actividad independiente

Lee las instrucciones y completa los códigos abajo. La idea es realizar los principales pasos asociados al entrenamiento y validación de un modelo clasificador. Por ahora, nos limitaremos a utilizar herramientas e ideas que hemos visto en clases. Progresivamente iremos añadiendo complejidad y nuevas herramientas a este flujo. 

Para este ejemplo utilizaremos una librería llamada sklearn.

### Cargamos los datos

In [30]:
general_path = './data/music_genre'
print(list(os.listdir(f'{general_path}/genres_original/')))

['pop', 'metal', 'disco', 'blues', 'reggae', 'classical', 'rock', 'hiphop', 'country', 'jazz']


In [31]:
data = pd.read_csv(f'{general_path}/features_3_sec.csv')
data = data.iloc[0:, 1:] 
data.head()

,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,3714.560359,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,3869.682242,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,3997.639160,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,3568.300218,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,3469.992864,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues


### Separamos en características y clases en este caso X,y

In [32]:
y = data['label'] # Género
X = data.loc[:, data.columns != 'label'] # Selecciona todas las columnas excepto las clases

In [33]:
y.value_counts()

label
blues        1000
jazz         1000
metal        1000
pop          1000
reggae       1000
disco         999
classical     998
hiphop        998
rock          998
country       997
Name: count, dtype: int64

In [34]:
# Extraemos los nombres de las columnas
cols = X.columns

### Primer Paso

Abre una nueva célula y observa tanto en $X$ como en $y$ los valores utilizando el método de esos objetos llamado ```describe()``` herededo desde ```pandas```. ¿Crees que es necesario escalar los datos?

In [35]:
X.describe()

,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
count,9990.0,9990.000000,9990.000000,9990.000000,9.990000e+03,9990.000000,9.990000e+03,9990.000000,9.990000e+03,9990.000000,...,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000
mean,66149.0,0.379534,0.084876,0.130859,2.676388e-03,2199.219431,4.166727e+05,2241.385959,1.182711e+05,4566.076592,...,1.448240,49.988755,-4.198706,51.962753,0.739943,52.488851,-2.497306,54.973829,-0.917584,57.322614
std,0.0,0.090466,0.009637,0.068545,3.585628e-03,751.860611,4.349644e+05,543.854449,1.013505e+05,1642.065335,...,5.735149,34.442816,5.677379,36.400669,5.181313,38.177120,5.111799,41.585677,5.253243,46.444212
min,66149.0,0.107108,0.015345,0.000953,4.379535e-08,472.741636,8.118813e+02,499.162910,1.183520e+03,658.336276,...,-26.850016,1.325786,-27.809795,1.624544,-20.733809,3.437439,-27.448456,3.065302,-35.640659,0.282131
25%,66149.0,0.315698,0.079833,0.083782,6.145900e-04,1630.680158,1.231961e+05,1887.455790,4.876553e+04,3378.311110,...,-2.227478,29.584894,-7.951722,29.863448,-2.516638,29.636197,-5.734123,30.496412,-4.004475,30.011365
50%,66149.0,0.384741,0.085108,0.121253,1.491318e-03,2208.628236,2.650692e+05,2230.575595,8.996072e+04,4631.377892,...,1.461623,41.702393,-4.443021,42.393583,0.733772,41.831377,-2.702366,43.435253,-1.030939,44.332155
75%,66149.0,0.442443,0.091092,0.176328,3.130862e-03,2712.581884,5.624152e+05,2588.340505,1.585674e+05,5591.634521,...,5.149752,59.274619,-0.726945,61.676964,3.888734,62.033906,0.514246,65.328602,2.216603,68.210421
max,66149.0,0.749481,0.120964,0.442567,3.261522e-02,5432.534406,4.794119e+06,3708.147554,1.235143e+06,9487.446477,...,39.144405,683.932556,34.048843,529.363342,36.970322,629.729797,31.365425,1143.230591,34.212101,910.473206


In [36]:
y.describe()

count      9990
unique       10
top       blues
freq       1000
Name: label, dtype: object

De la libería ```sklearn``` importamos ```preprocessing```. Utiliza ```help(preprocessing.MinMaxScaler())``` para entender como utilizar el método

In [37]:
help(preprocessing.MinMaxScaler())

Help on MinMaxScaler in module sklearn.preprocessing._data object:

class MinMaxScaler(sklearn.base.OneToOneFeatureMixin, sklearn.base.TransformerMixin, sklearn.base.BaseEstimator)
 |  MinMaxScaler(feature_range=(0, 1), *, copy=True, clip=False)
 |  
 |  Transform features by scaling each feature to a given range.
 |  
 |  This estimator scales and translates each feature individually such
 |  that it is in the given range on the training set, e.g. between
 |  zero and one.
 |  
 |  The transformation is given by::
 |  
 |      X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
 |      X_scaled = X_std * (max - min) + min
 |  
 |  where min, max = feature_range.
 |  
 |  This transformation is often used as an alternative to zero mean,
 |  unit variance scaling.
 |  
 |  `MinMaxScaler` doesn't reduce the effect of outliers, but it linearily
 |  scales them down into a fixed range, where the largest occuring data point
 |  corresponds to the maximum value and the smallest one

Luego, instancia el escalador de datos con ```min_max_scaler = preprocessing.MinMaxScaler()```. 

In [38]:
### Aqui instancia el escalador de datos
min_max_scaler = preprocessing.MinMaxScaler()

Finalmente, escala tus datos a través del método de clase de tu ```min_max_scaler``` llamado ```.fit_transform()```
Este método recibe como entrada tus datos $X$

In [39]:
### Escala tus datos aquí
np_scaled = min_max_scaler.fit_transform(X)
### Convertimos la data en una tabla estructurada
X = pd.DataFrame(np_scaled, columns = cols)

Utiliza nuevamente la función ```describe()``` sobre $X$ para confirmar el escalamiento de los datos

In [40]:
X.describe()

,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
count,9990.0,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000,...,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000
mean,0.0,0.424094,0.658319,0.294161,0.082058,0.348095,0.086759,0.542920,0.094888,0.442597,...,0.428798,0.071290,0.381694,0.095385,0.372135,0.078320,0.424239,0.045527,0.497090,0.062669
std,0.0,0.140831,0.091239,0.155216,0.109937,0.151591,0.090744,0.169479,0.082134,0.185983,...,0.086904,0.050458,0.091780,0.068975,0.089791,0.060957,0.086915,0.036473,0.075205,0.051027
min,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.0,0.324718,0.610570,0.187559,0.018842,0.233465,0.025532,0.432627,0.038560,0.308069,...,0.373100,0.041399,0.321023,0.053509,0.315700,0.041832,0.369204,0.024059,0.452898,0.032663
50%,0.0,0.432199,0.660514,0.272410,0.045723,0.349992,0.055130,0.539552,0.071945,0.449993,...,0.429000,0.059151,0.377745,0.077252,0.372028,0.061304,0.420753,0.035407,0.495467,0.048396
75%,0.0,0.522026,0.717174,0.397122,0.095993,0.451600,0.117164,0.651040,0.127544,0.558754,...,0.484886,0.084893,0.437818,0.113792,0.426703,0.093561,0.475444,0.054609,0.541958,0.074631
max,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Ahora divide en entrenamiento y test el conjunto de datos. Utiliza ```help(train_test_split)``` y divide los datos con un ```test_size=0.3``` y un ```random_state = 42``` 

In [41]:
from sklearn.model_selection import train_test_split

In [42]:
help(train_test_split)

Help on function train_test_split in module sklearn.model_selection._split:

train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)
    Split arrays or matrices into random train and test subsets.
    
    Quick utility that wraps input validation,
    ``next(ShuffleSplit().split(X, y))``, and application to input data
    into a single call for splitting (and optionally subsampling) data into a
    one-liner.
    
    Read more in the :ref:`User Guide <cross_validation>`.
    
    Parameters
    ----------
    *arrays : sequence of indexables with same length / shape[0]
        Allowed inputs are lists, numpy arrays, scipy-sparse
        matrices or pandas dataframes.
    
    test_size : float or int, default=None
        If float, should be between 0.0 and 1.0 and represent the proportion
        of the dataset to include in the test split. If int, represents the
        absolute number of test samples. If None, the value is set to

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=  42)

Cargamos los modelos revisados en clase para clasificar y las métricas para evaluar

In [44]:
from sklearn.linear_model import LogisticRegression #regresión logística
from sklearn.neural_network import MLPClassifier    #perceptrón multicapa
from sklearn.metrics import confusion_matrix, accuracy_score # matriz de confusión y accuracy

Completa la siguiente clase con la información faltante

Observa que ```hidden_layer_sizes``` representa un modelo de dos capas una con dimension 500 y otra con dimension 10. La dimensión de entrada dependerá del número de características, en este caso 59 características dadas por el dataset. 
Por lo tanto las matrices serán de $59 \times 500$ y $500 \times 10$. La ùltima capa será de $10 \times \text{numero de clases}$

In [45]:
class MiPrimeraRed:

    def __init__(self, hidden_layer_sizes=(500, 10), alpha=1e-3, solver='sgd', max_iter = 1000):
        ### COMPLETA ESTOS ATRIBUTOS
        self.hidden_layer_sizes = hidden_layer_sizes  #número de capas escondidas
        self.alpha = alpha #learning rate
        self.solver = solver #optimizador, por ahora utilizaremos el que conocemos Gradiente Estocástico
        self.max_iter = max_iter
        ### INICIALIZAMOS EL PERCEPTRON MULTICAPA
        self.nn = MLPClassifier(solver= self.solver, 
                                alpha=self.alpha, 
                                hidden_layer_sizes = self.hidden_layer_sizes, 
                                max_iter = self.max_iter ,
                                random_state=1)
        
    def train_model(self, X_train, y_train):
        try:
            self.nn.fit(X_train,y_train)
            print("Modelo entrenado exitosamente!")
        except Exception as e:
            print(f"Algo ha ocurrido al momento de entrenar: {e}")

    def measuring_model_accuracy(self, X_test, y_test):
        # confirmamos que el modelo ya fue entrenado para poder medir accuracy
        if hasattr(self.nn, 'coefs_'):
            preds = self.nn.predict(X_test)
            print('Accuracy', accuracy_score(y_test, preds), '\n')
            print("Matriz de Confusión")
            print(confusion_matrix(y_test, preds))
        else:
            print('El modelo no ha sido entrenado, no es posible medir accuracy')
            
    def summary(self):
        # Print model configuration
        print("Model Configuration:")
        print(f"Hidden Layer Sizes: {self.hidden_layer_sizes}")
        print(f"Alpha (Learning Rate): {self.alpha}")
        print(f"Solver (Optimizer): {self.solver}")
        print(f"Max Iterations: {self.max_iter}")

        # Check if the model has been trained
        if hasattr(self.nn, 'coefs_'):
            print("\nTraining Summary:")
            
            # Print the number of iterations
            print(f"Number of Iterations: {self.nn.n_iter_}")
            
            # Print the loss
            print(f"Loss: {self.nn.loss_:.4f}")
            
            # Print the configuration of each layer
            for i, (coef, intercept) in enumerate(zip(self.nn.coefs_, self.nn.intercepts_), start=1):
                print(f"Layer {i}:")
                print(f" - Weights shape: {coef.shape}")
                print(f" - Biases shape: {intercept.shape}")
            
            # Optionally, you could add performance metrics (e.g., accuracy, confusion matrix, classification report)
            # You would need to store X_test and y_test as attributes of the class after measuring_model_accuracy is called
            # to be able to reference them here.
        else:
            print('The model has not been trained yet.')



Ahora instancia tu modelo, entrenalo y evalúalo utilizando los métodos dentro de las clases!

In [46]:
mi_modelo = MiPrimeraRed()

In [47]:
mi_modelo.train_model(X_train,y_train)

Modelo entrenado exitosamente!


/Users/rodolfolobocarrasco/.pyenv/versions/3.11.6/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [48]:
mi_modelo.measuring_model_accuracy(X_test,y_test)

Accuracy 0.7380714047380714 

Matriz de Confusión
[[230   2  17   8   3  17  20   0  10  12]
 [  1 290   2   0   0  14   0   0   0   1]
 [ 14   2 183   7   0  20   2  13   6  39]
 [  2   3  10 190  12   5   9  20  20  30]
 [ 12   2   4  13 221   0  11  15  30   3]
 [  4  26   6   0   0 243   0   2   5   0]
 [  9   0   4   4   6   1 265   0   1  13]
 [  2   0   9   8   3   0   0 235   7   3]
 [  6   1  15  10  25   5   3  12 231   8]
 [ 16   7  34  46   3  20  19  14  17 124]]


In [49]:
mi_modelo.summary()

Model Configuration:
Hidden Layer Sizes: (500, 10)
Alpha (Learning Rate): 0.001
Solver (Optimizer): sgd
Max Iterations: 1000

Training Summary:
Number of Iterations: 1000
Loss: 0.7041
Layer 1:
 - Weights shape: (58, 500)
 - Biases shape: (500,)
Layer 2:
 - Weights shape: (500, 10)
 - Biases shape: (10,)
Layer 3:
 - Weights shape: (10, 10)
 - Biases shape: (10,)


Observe la clase original de ```MLPClassifier``` [Link](https://github.com/scikit-learn/scikit-learn/blob/093e0cf14aff026cca6097e8c42f83b735d26358/sklearn/neural_network/_multilayer_perceptron.py#L382)

In [50]:
mi_modelo.nn

MLPClassifier(alpha=0.001, hidden_layer_sizes=(500, 10), max_iter=1000,
              random_state=1, solver='sgd')

In [13]:
class Solution(object):
    def twoSum(self, nums, target):
        """
        :type nums: List[int]
        :type target: int
        :rtype: List[int]
        """
        response = {}
        key = 0
        for i, ref in enumerate(nums):
            for j, num in enumerate(nums):
                if i != j:
                    candidate_target = num + ref
                    print("candidate target", candidate_target)
                    if candidate_target == target:
                        print(candidate_target == target)
                        print(i,j)
                        if set([i,j]) not in response.values():
                            response[key]=set([i,j])
                            key =+ 1
                            print("response", response)
        output = list()
        
        for e in response.values():
            output.append(e)
        return output

nums = [2,7,11,15]
target = 9
my_class = Solution()
output = my_class.twoSum(nums, target)

candidate target 9
True
0 1
response {0: {0, 1}}
candidate target 13
candidate target 17
candidate target 9
True
1 0
candidate target 18
candidate target 22
candidate target 13
candidate target 18
candidate target 26
candidate target 17
candidate target 22
candidate target 26


In [14]:
output

[{0, 1}]